In [1]:
from syft.core.hooks import TorchHook
from syft.core.hooks import torch
from syft.core.workers import VirtualWorker
from torch.autograd import Variable
hook = TorchHook()

def relu(x):
    """Rectified linear activation"""
    return torch.clamp(x, min=0.)

def linear(x, w):
    """Linear transformation of x by w"""
    return x.mm(w)

x = Variable(torch.FloatTensor([[1,1],[2,2]]), requires_grad=True)
y = Variable(torch.FloatTensor([[1,1],[2,2]]), requires_grad=True)

z = linear(x, y)

# print(z.data.is_pointer)

z = relu(z)

Hooking into Torch...
Overloading complete.


In [7]:
z

Variable containing:
 3  3
 6  6
[torch.FloatTensor of size 2x2]

False


Variable containing:
 3  3
 6  6
[torch.FloatTensor of size 2x2]

In [ ]:


#print(z)


AttributeError: 'torch.FloatTensor' object has no attribute 'is_pointer'

In [1]:

import json
# hook torch and create a local worker
hook = TorchHook()
local = hook.local_worker

Hooking into Torch...
Overloading complete.


In [2]:
# lets create a remote worker
remote = VirtualWorker(hook=hook,id=1)
local.add_worker(remote)
remote

In [3]:
# we can get the worker using it's id (1)
local.get_worker(1)

In [4]:
# or we can get the worker by passing in the worker
local.get_worker(remote)

In [2]:
x = torch.FloatTensor([1,2,3,4,5]).send(remote)
x

[torch.FloatTensor - Locations:[<syft.core.workers.VirtualWorker object at 0x10f871c18>]]

In [3]:
x.id

3214169934

In [4]:
remote.get_obj(x.id)

[torch.FloatTensor - Locations:[1]]

In [2]:
torch_type = hook.types_guard('torch.FloatTensor')
x = torch_type([1,2,3,4,5])
x


 1
 2
 3
 4
 5
[torch.FloatTensor of size 5]

In [3]:
x.send(remote)

[torch.FloatTensor - Locations:[<syft.core.workers.VirtualWorker object at 0x117435588>]]

In [4]:
x

[torch.FloatTensor - Locations:[<syft.core.workers.VirtualWorker object at 0x117435588>]]

In [5]:
y = x + x

In [6]:
y.owners[0]

In [7]:
y.get()


  2
  4
  6
  8
 10
[torch.FloatTensor of size 5]

In [21]:
x = Var(torch.ones(2,2),requires_grad=True).send_(remote)
x2 = Var(torch.ones(2,2)*2,requires_grad=True).send_(remote)

In [22]:
y = x * x2

In [23]:
y.sum().backward()

In [24]:
x_remote = remote._objects[x2.id]

In [25]:
y.get()

Variable containing:
 2  2
 2  2
[torch.FloatTensor of size 2x2]

In [26]:
x.get()

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [27]:
x.grad

Variable containing:
 2  2
 2  2
[torch.FloatTensor of size 2x2]

In [28]:
x2.get()

Variable containing:
 2  2
 2  2
[torch.FloatTensor of size 2x2]

In [29]:
x2.grad

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]